In [8]:
import geopandas as gpd
import pandas as pd
from ipyleaflet import Map, GeoJSON, WidgetControl
import ipywidgets as widgets

# Shapefile laden
world = gpd.read_file("./data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")
europe = world[world['CONTINENT'] == 'Europe']

# CSV laden und bereinigen
df = pd.read_csv("./data/assets/Co2_Stats.csv", sep='\t', encoding='utf-16')
df = df.rename(columns={'Country': 'ADMIN', 'Jahr von Date': 'year', 't CO2 equivalent': 'co2'})
df['co2'] = df['co2'].str.replace('.', '', regex=False).astype(float)
df['year'] = df['year'].astype(int)

# Länder-Namen ggf. anpassen (Mapping vor Pivot!)
country_name_map = {
    "Czechia": "Czech Republic",
    "Slovakia": "Slovak Republic",
    "Bosnia and Herzegovina": "Bosnia and Herz.",
    "North Macedonia": "Macedonia",
    "Russia": "Russian Federation"
}
df['ADMIN'] = df['ADMIN'].replace(country_name_map)

# Pivot: eine Zeile pro Land, Spalten für Jahre
emissions_df = df.pivot(index='ADMIN', columns='year', values='co2').reset_index()

# Merge mit Shapefile
europe_emissions = europe.merge(emissions_df, how='left', on='ADMIN').fillna(0)

# GeoJSON-Style-Funktion
def get_geojson(year):
    def style_callback(feature):
        admin = feature['properties']['ADMIN']
        value = emissions_df.loc[emissions_df['ADMIN'] == admin, year].values
        emission = value[0] if len(value) else 0
        max_emission = emissions_df[year].max() if emissions_df[year].max() > 0 else 1
        intensity = min(emission / max_emission, 1.0)
        color = f"rgba(255, 0, 0, {intensity})"
        return {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7,
        }
    return GeoJSON(
        data=europe_emissions.__geo_interface__,
        style_callback=style_callback,
        name=f"CO₂ Emissionen {year}"
    )

# Map und Dropdown
center = [54.0, 15.0]
m = Map(center=center, zoom=4)

# Jahre aus den Daten dynamisch holen
years = sorted([col for col in emissions_df.columns if isinstance(col, int)])

layer_ref = [get_geojson(years[0])]
m.add_layer(layer_ref[0])

dropdown = widgets.Dropdown(
    options=[(str(y), y) for y in years],
    value=years[0],
    description='Jahr:',
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        m.remove_layer(layer_ref[0])
        new_layer = get_geojson(change['new'])
        m.add_layer(new_layer)
        layer_ref[0] = new_layer

dropdown.observe(on_change)

control = WidgetControl(widget=dropdown, position='topright')
m.add_control(control)

m

/var/folders/7g/n_c_1w8j6nq8cmbcbnl7mpcc0000gn/T/ipykernel_7350/4238406504.py:30: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  europe_emissions = europe.merge(emissions_df, how='left', on='ADMIN').fillna(0)


Map(center=[54.0, 15.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [10]:
import geopandas as gpd

# Shapefile laden
world = gpd.read_file("./data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")
europe = world[world['CONTINENT'] == 'Europe']

# Englische Ländernamen ausgeben (Spalte 'ADMIN')
geo_names = sorted(europe['ADMIN'].unique())
print("Englische Ländernamen aus der Geodaten-Spalte 'ADMIN':")
for name in geo_names:
    print(name)
    

Englische Ländernamen aus der Geodaten-Spalte 'ADMIN':
Albania
Austria
Belarus
Belgium
Bosnia and Herzegovina
Bulgaria
Croatia
Czechia
Denmark
Estonia
Finland
France
Germany
Greece
Hungary
Iceland
Ireland
Italy
Kosovo
Latvia
Lithuania
Luxembourg
Moldova
Montenegro
Netherlands
North Macedonia
Norway
Poland
Portugal
Republic of Serbia
Romania
Russia
Slovakia
Slovenia
Spain
Sweden
Switzerland
Ukraine
United Kingdom
